## Problem

In OpenQAOA there are several problems that are implemented in order to perform the Ising model, in this example we will address a specific case of  the [Partition problem](https://en.wikipedia.org/wiki/Partition_problem), this problem is to determine whether a given set can be partitioned into two subsets such that the sum of elements in both subsets is the same. 

### Classical Method

The classical idea to solve this problem following the next steps:

- Calculate sum of the array. If sum is odd, there can not be two subsets with equal sum, so return false. 
- If sum of array elements is even, calculate sum/2 and find a subset of array with sum equal to sum/2. 

<div class="alert alert-block alert-warning"> 

For the input in this example we can obtain for the first step that the sum of **[4, 3, 3, 2]** is  *12*, therefore this step is correct. 

For step two, we have the following subsets:

</div>


<table  style="width:100%;color: white;">
    <thead >
        <tr style=" background-color: #2f3974;">
            <th>Case </th>
            <th> $s_1$</th>
            <th> $s_2$</th>
            <th> $\sum(s_1)$</th>
            <th> $\sum(s_2)$</th>
            <th> Output states</th>
        </tr>
    </thead>
    <tbody>
        <tr style=" background-color: #3a6486;">
            <td>1</td>
            <td>[4,3,3,2]</td>
            <td>[]</td>
            <td>12</td>
            <td>0</td>
            <td>[[1,1,1,1],[0,0,0,0]]</td>
        </tr>
        <tr style=" background-color: #3a6486;">
            <td>2</td>
            <td>[4,3,3]</td>
            <td>[2]</td>
            <td>10</td>
            <td>2</td>
            <td>[[1,1,1,0],[0,0,0,1]]</td>
        </tr>
        <tr style=" background-color: #3a6486;"> 
            <td>3</td>
            <td>[4,3]</td>
            <td>[3,2]</td>
            <td>7</td>
            <td>5</td>
            <td>[[1,1,0,0],[0,0,1,1]],[[1,0,1,0],[0,1,0,1]]</td>
        </tr>
        <tr style=" background-color: #4fbeae;">
            <td>4</td>
            <td>[3,3]</td>
            <td>[4,2]</td>
            <td>6</td>
            <td>6</td>
            <td>[[0,1,1,0],[1,0,0,1]]</td>
        </tr>
    </tbody>
 </table>

<center>Table 1. Description step 2 of classical solution</center>


Figure 1 shows that the case 4 fulfills two subsets with the value in the middle and therefore it is possible to perform the answer considering the partition of the indices 0 and 3, as well as the indices 2 and 3.

### Design  the problem

The necessary dependencies to build the problem have to be initialized, in order to display the graphs of the results and the necessary OpenQAOA objects and methods to generate the quantum algorithm.

In [1]:
# Methods to use in the  notebook
%load_ext autoreload
%autoreload 2
from IPython.display import clear_output

# Libaries to manipulate, print, plot, and save the data.
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Random generator for the problem
from random import seed,randrange, randint

# OpenQAOA libraries to design, execute the algorithm in different backends and optimisers
from openqaoa.problems.problem import NumberPartition
from openqaoa.workflows.optimizer import QAOA 
from openqaoa.devices import create_device
from openqaoa.optimizers.qaoa_optimizer import available_optimizers
from openqaoa.utilities import ground_state_hamiltonian

Using the method random we can generate a random list of four elements, from which the Partition problem is to be implemented.

In [2]:
# To reproduce the code and make this a problem that has optimal states in the bipartition problem,
# the following seed is used
seed(9)

# Generate four random number with a range between (1,5) callend the variable in_list
in_list = [randrange(1, 5, 1) for i in range(4)]
in_list

[4, 3, 3, 2]

####  Number Partition problem to QAOA

In order to work the problem you want to design you must convert it to Quadratic Unconstrained Binary Optimization (QUBO) format. That is possible using the Class *NumberPartition*, where this has one parameter, an integer list, one object of this class has the method `get_pubo_problem()`, that translates the problem into a QUBO problem.

In [3]:
qubo = NumberPartition(in_list).get_qubo_problem()
qubo.asdict()

{'terms': [[0, 1], [0, 2], [0, 3], [1, 2], [1, 3], [2, 3]],
 'weights': [24.0, 24.0, 16.0, 18.0, 12.0, 12.0],
 'constant': 38,
 '_n': 4}

Implement the QAOA quantum algorithm  from the OpenQAOA implementation

In [4]:
# In OpenQAOA you can use different devices either local 
# or from different cloud backends such as Qiskit, Pyquil.
sim = create_device(location='local', name='vectorized')

# Init the object QAOA and certain of its properties can be modified
q = QAOA()
q.set_device(sim)
q.compile(qubo) 

# Execute the quantum algorithm
q.optimize()

#### Results

You can print different things from the QAOA object in the section `q.results` as:

and check the `solutions_bitstrings` and the exactly answer with `ground_state_hamiltonian`

In [5]:
print("solution bitstring: ",q.results.most_probable_states['solutions_bitstrings'])
print("ground state: ",ground_state_hamiltonian(q.cost_hamil)[1])

solution bitstring:  ['0110', '1001']
ground state:  ['0110', '1001']


### Try it yourself

Consider the procces from the previous example we analize different results using all the optimizers that OpenQAOA support, this divide in *'scipy'* and *'custom_scipy_gradient'*, as:

In [6]:
optimisers = available_optimizers()['scipy']+ available_optimizers()['custom_scipy_gradient']
optimisers

['nelder-mead',
 'powell',
 'cg',
 'bfgs',
 'newton-cg',
 'l-bfgs-b',
 'tnc',
 'cobyla',
 'slsqp',
 'trust-constr',
 'dogleg',
 'trust-ncg',
 'trust-exact',
 'trust-krylov',
 'vgd',
 'newton',
 'rmsprop',
 'natural_grad_descent',
 'spsa']

Using 19 optimisers, for all those you can choose the method to compute the Jacobian and Hessian, for Jacobian you can consider all the  methods and Hessian only *finite_difference*.

Define the Jacobian list:

In [7]:
jac = ['finite_difference', 'param_shift','grad_spsa', 'stoch_param_shift']

Define the Hessian value:

In [8]:
hess = jac[0]